<a href="https://colab.research.google.com/github/mmsamiei/untitleds/blob/main/digikala_faq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle
!pip install -q transformers

     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 1.1MB 19.5MB/s 
     |████████████████████████████████| 890kB 26.7MB/s 
     |████████████████████████████████| 3.0MB 26.3MB/s 


In [ ]:
from google.colab import files

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                               title                                                 size  lastUpdated          downloadCount  
----------------------------------------------------------------  --------------------------------------------------  ------  -------------------  -------------  
christianlillelund/donald-trumps-rallies                          Donald Trump's Rallies                               720KB  2020-09-26 10:25:08            559  
heeraldedhia/groceries-dataset                                    Groceries dataset                                    257KB  2020-09-17 04:36:08           2910  
andrewmvd/trip-advisor-hotel-reviews                              Trip Advisor Hotel Reviews                             5MB  2020-09-30 08:31:20           1679  
balraj98/stanford-background-dataset                              Stanford Background Dataset                           17MB  2020-09-26 12:57:59            175  
nehaprabhavalkar/india

In [ ]:
!kaggle datasets download -d miladfa7/persian-wikipedia-dataset

 96% 201M/209M [00:04<00:00, 40.7MB/s]
100% 209M/209M [00:04<00:00, 44.5MB/s]


In [ ]:
!unzip /content/persian-wikipedia-dataset.zip

Archive:  /content/persian-wikipedia-dataset.zip
  inflating: Persian-WikiText-1.txt  
  inflating: Persian-WikiText-2.txt  
  inflating: Persian-WikiText-3.txt  
  inflating: Persian-WikiText-4.txt  
  inflating: Persian-WikiText-5.txt  
  inflating: Persian-WikiText-6.txt  
  inflating: Persian-WikiText-7.txt  
  inflating: Persian-WikiText-8.txt  
  inflating: Persian-WikiText-9.txt  


In [2]:
from transformers import AutoConfig, AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("m3hrdadfi/albert-fa-base-v2")

In [3]:
import numpy as np
from tqdm.auto import tqdm
freqs = np.zeros(tokenizer.vocab_size)

In [ ]:
wiki_file = open('Persian-WikiText-1.txt')

In [ ]:
sentences = wiki_file.readlines()

In [ ]:
for sentence in tqdm(sentences):
  tokens = tokenizer(sentence)['input_ids']
  for token in tokens:
    freqs[token] += 1

In [ ]:
np.save('scores.npy', scores)
l = np.load('scores.npy')

In [8]:
scores = np.load('/content/scores.npy')

In [40]:
for a in open('stopwords.txt').readlines():
  for b in tokenizer.encode(a.strip())[1:-1]:
    scores[b] = 0.001

# Another Part


In [42]:
from numpy.linalg import norm
from scipy.spatial.distance import cdist
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoConfig, AutoModel, AutoTokenizer
import numpy as np
import torch
import json
config = AutoConfig.from_pretrained("m3hrdadfi/albert-fa-base-v2")
tokenizer = AutoTokenizer.from_pretrained("m3hrdadfi/albert-fa-base-v2")
model = AutoModel.from_pretrained("m3hrdadfi/albert-fa-base-v2")

In [43]:
def argmax(iterable):
    return max(enumerate(iterable), key=lambda x: x[1])[0]

In [133]:
def find_similar(query:str, values:[str]):
    with torch.no_grad():
        sentences = values.copy()
        sentences.append(query)
        sentences_tknzed = tokenizer(sentences, return_tensors='pt', padding='longest',max_length=25)
        sentences_ids = sentences_tknzed['input_ids']
        attention_mask = sentences_tknzed['attention_mask']
        sentences_embeddings = model(input_ids=sentences_ids, attention_mask=attention_mask, return_dict=True).last_hidden_state[:,:,:].detach().numpy()

    query_embedding = sentences_embeddings[-1:,:,:][0]
    values_embedding = sentences_embeddings[:-1,:,:]
    query_mask = attention_mask[-1,:].detach().numpy()
    query_len = query_mask.sum()
    #query_mask[0] = 0
    values_mask = attention_mask[:-1].detach().numpy()
    query_embedding = query_embedding[1:query_len-1,:]
    f_scores = []
    for value_embedding, value_mask, sentence_ids in zip(values_embedding, values_mask, sentences_ids):
      value_len = value_mask.sum()
      value_embedding = value_embedding[1:value_len-1,:]
      value_weights = scores[sentence_ids[1:value_len-1]]
      query_ids = sentences_ids[-1]
      query_weights = scores[query_ids[1:query_mask.sum()-1]]
      # print(query_weights)
      similarity = cosine_similarity(value_embedding, query_embedding)
      recall_list = similarity.max(axis=1)
      # print(recall_list)
      precision_list = similarity.max(axis=0)
      recall = ((recall_list*value_weights).sum()/value_weights.sum())
      precision = ((precision_list*query_weights).sum()/query_weights.sum())
      # recall = recall_list.mean()
      # precision = precision_list.mean()
      f_score = 2 * (precision * recall) / (precision + recall)
      # print(recall_list)
      # print(value_weights)
      # print(precision_list)
      #print(query_weights)
      # print(recall)
      f_scores.append(f_score)
      # print("-------")

    
    # print(f_scores)
    best_i = argmax(f_scores)
    best_score = f_scores[best_i]
    
    #values_scores = sorted(zip(values, f_scores), key = lambda t: -t[1])
    #print(values_scores[:5])

    return sentences[best_i], best_i, best_score



In [134]:
questions = [row['question'].strip() for row in json.load(open('/content/digikala-faq-qas.json'))]

In [135]:
for i in tokenizer.encode(query):
  print(tokenizer.decode(i))

[CLS]
ارسال
اکسپرس
یعنی
چه
؟
[SEP]


In [144]:
query = ' چطور میتوانم رمز عبور خود را ریست کنم؟'
values = [
'چطور میتوانم سفارشم را لغو کنم ؟',
 'چطور میتوانم امتیاز بگیریم؟',
]
find_similar(query, questions)

[('چگونه می توانم رمز عبور کیف پول خود را غیرفعال کنم؟', 0.8043483992134227), ('آیا میتوانم کارت هدیه را مرجوع کنم؟', 0.7397319102078168), ('آیا میتوانم کارت هدیه را مرجوع کنم؟', 0.7397319102078168), ('چگونه برای کیف پول خود رمز عبور تعیین کنم؟', 0.7197440507069525), ('رمز عبور خود را فراموش کردم، چطور میتوانم از طریق نسخه وب رمز خود را بازیابی کنم؟', 0.7148922558901077)]


('چگونه می توانم رمز عبور کیف پول خود را غیرفعال کنم؟', 73, 0.8043483992134227)

##another part

In [ ]:
!pip install googletrans

In [ ]:
from googletrans import Translator
import nlpaug.augmenter.word as naw
translator = Translator()

In [ ]:
initial_text = query
aug = naw.SynonymAug(aug_src='wordnet')
translated = translator.translate(initial_text).text
augmented_text = aug.augment(translated)
back_translated = translator.translate(augmented_text, dest='persian').text
back_translated

'ید چگونه می تواند کارت وقف دریافت کند؟'

In [ ]:
!pip install -q  requests nlpaug nltk>=3.4.5